# Fine-tuning the top layers of the pre-trained VGG16
We can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. 

**Remark** Although Keras and tf.keras in most cases are compatible, but sometimes you will have unexpected errors. Most of the suggestions on the website are for Keras, some of them will not work for tf.keras.

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input

# path to the model weights files.
weights_path = 'vgg16_weights_notop.h5'
top_model_weights_path = 'fc_model.h5'

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/dogs-vs-cats-small/train'
validation_data_dir = 'data/dogs-vs-cats-small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16

img_input_shape = (img_width,img_height,3)

# build the VGG16 network
vgg_conv = applications.VGG16(weights='imagenet', include_top=False,input_shape=img_input_shape)
print('Model loaded.')

# set the first 21 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
    
for layer in vgg_conv.layers:
    print(layer, layer.trainable)
          
vgg_conv.summary()

Using TensorFlow backend.


Model loaded.
<keras.engine.input_layer.InputLayer object at 0x7f25ec52fe10> False
<keras.layers.convolutional.Conv2D object at 0x7f255ff7eb38> False
<keras.layers.convolutional.Conv2D object at 0x7f255ff7e9b0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f255ffab400> False
<keras.layers.convolutional.Conv2D object at 0x7f255ffab978> False
<keras.layers.convolutional.Conv2D object at 0x7f255f768160> False
<keras.layers.pooling.MaxPooling2D object at 0x7f255f70a2e8> False
<keras.layers.convolutional.Conv2D object at 0x7f255f70acc0> False
<keras.layers.convolutional.Conv2D object at 0x7f255f6be0f0> False
<keras.layers.convolutional.Conv2D object at 0x7f255f6de4e0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f255f67f2e8> False
<keras.layers.convolutional.Conv2D object at 0x7f255f67fcc0> False
<keras.layers.convolutional.Conv2D object at 0x7f256b9090b8> False
<keras.layers.convolutional.Conv2D object at 0x7f255f6530b8> False
<keras.layers.pooling.MaxPooling2D object at 0

In [2]:
# build a classifier model to put on top of the convolutional model

total_model = Sequential()
total_model.add(vgg_conv)
total_model.add(Flatten())
total_model.add(Dense(256, activation='relu'))
total_model.add(Dropout(0.5))
total_model.add(Dense(1, activation='sigmoid'))
total_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [3]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
total_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
total_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/home/lyealy/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/lyealy/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=125, validation_steps=800)`


Epoch 1/5
125/125 [==============================] - 41s 326ms/step - loss: 0.6133 - acc: 0.6675 - val_loss: 0.4192 - val_acc: 0.8462
Epoch 2/5
125/125 [==============================] - 40s 317ms/step - loss: 0.4148 - acc: 0.8205 - val_loss: 0.3141 - val_acc: 0.8700
Epoch 3/5
125/125 [==============================] - 39s 312ms/step - loss: 0.3358 - acc: 0.8580 - val_loss: 0.2618 - val_acc: 0.8912
Epoch 4/5
125/125 [==============================] - 40s 318ms/step - loss: 0.2941 - acc: 0.8795 - val_loss: 0.2477 - val_acc: 0.9000
Epoch 5/5
125/125 [==============================] - 39s 312ms/step - loss: 0.2578 - acc: 0.8925 - val_loss: 0.2162 - val_acc: 0.9087
